### The Perron-Frobenius Theorem implementation


In [ ]:
from scipy.sparse import dok_matrix
import pandas as pd
import numpy as np
from scipy.sparse.linalg import eigs
from scipy.sparse import dok_matrix
from scipy.linalg import solve

In [ ]:
df = pd.read_csv('matches.csv', delimiter=',')

Constructing an adjency matrix for teams of season 

In [ ]:
results_dict22, results_dict21 = {}, {}
teams22, teams21 = [], []

for index, row in df.iterrows():
    team = row['team']
    opponent = row['opponent']
    result = row['result']

    if row['season'] == 2022:
        if team not in teams22: teams22.append(team)
        if team in results_dict22:
            results_dict22[team].append({'opponent': opponent, 'result': result})
        else:
            results_dict22[team] = [{'opponent': opponent, 'result': result}]
    else:
        if team not in teams21: teams21.append(team)
        if team in results_dict21:
            results_dict21[team].append({'opponent': opponent, 'result': result})
        else:
            results_dict21[team] = [{'opponent': opponent, 'result': result}]


def trimmed(team_key, diict, tms):
    return (team_key, [entry for entry in diict[team_key] if entry['opponent'] in tms])

teams_dict22 = dict([trimmed(team, results_dict22, teams22) for team in teams22])
teams_dict21 = dict([trimmed(team, results_dict21, teams21) for team in teams21])


def create_matrix(tms, tms_dict):
    n = len(tms)
    M = dok_matrix((n,n))
    for i in range(n):
        team = tms[i]
        games = tms_dict[team]
        for game in games:
            opponent = game['opponent']
            win_or_loss = game['result']
            if win_or_loss == "W":
                j = tms.index(opponent)
                M[i,j] = M[i,j] + 1 
            if win_or_loss == "L":
                j = tms.index(opponent)
                M[j,i] = M[j,i] + 1
    return M
M21 = create_matrix(teams21, teams_dict21)
M22 = create_matrix(teams22, teams_dict22)
# print('SEASON 2022')
# print(teams22)
# print(M22.toarray().astype(int))
# print('SEASON 2021')
# print(teams21)
# print(M21.toarray().astype(int))

Adjency matrix for teams of season 20/21 EPL:

In [ ]:
print(teams21)
M21.toarray().astype(int)

['Manchester City', 'Manchester United', 'Liverpool', 'Chelsea', 'Leicester City', 'West Ham United', 'Tottenham Hotspur', 'Arsenal', 'Leeds United', 'Everton', 'Aston Villa', 'Newcastle United', 'Wolverhampton Wanderers', 'Crystal Palace', 'Southampton', 'Brighton and Hove Albion', 'Burnley', 'Fulham', 'West Bromwich Albion', 'Sheffield United']


array([[0, 0, 2, 2, 2, 1, 1, 4, 0, 4, 4, 2, 2, 4, 4, 1, 4, 4, 1, 2],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 0, 2, 1, 0, 0],
       [0, 1, 0, 2, 2, 2, 2, 4, 2, 0, 2, 0, 2, 4, 2, 0, 2, 0, 1, 2],
       [2, 0, 2, 0, 2, 2, 1, 0, 2, 2, 0, 2, 0, 4, 0, 1, 4, 4, 0, 2],
       [2, 1, 2, 2, 0, 0, 1, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2],
       [0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 2, 0, 0, 1, 1, 0, 2, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 2, 1, 0, 0],
       [0, 1, 0, 4, 2, 1, 1, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2],
       [2, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 2, 4, 0, 4, 4, 2, 2],
       [0, 0, 2, 2, 2, 1, 1, 4, 2, 0, 0, 0, 2, 2, 2, 1, 0, 2, 2, 1],
       [0, 0, 2, 2, 2, 0, 1, 4, 2, 2, 0, 1, 1, 2, 2, 0, 0, 4, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 0, 2, 2, 2],
       [0, 0, 2, 0, 0, 0, 0, 0, 0,

Finding the eigensystem and building a ranking table:

In [ ]:
val,vec = eigs(M21, which='LM', k=1)
vec = np.ndarray.flatten(abs(vec))
sorted_indices = vec.argsort()
ranked1 = [(teams21[i], vec[i]) for i in sorted_indices]
ranked1.reverse()
ranked1

[('Manchester City', 0.4613666100654865),
 ('Leicester City', 0.32039824069263784),
 ('Chelsea', 0.31784111819233385),
 ('Liverpool', 0.3121575251730876),
 ('Aston Villa', 0.29765497036018046),
 ('Leeds United', 0.2897087541661704),
 ('Everton', 0.27265119797328236),
 ('Arsenal', 0.23510144865311716),
 ('Southampton', 0.17841637953728245),
 ('Burnley', 0.17656567962621578),
 ('West Ham United', 0.15728947443755562),
 ('Tottenham Hotspur', 0.14043099284464003),
 ('Manchester United', 0.13423059031119258),
 ('Crystal Palace', 0.1271772799815889),
 ('Wolverhampton Wanderers', 0.10375765494052105),
 ('Fulham', 0.10367997807707408),
 ('Brighton and Hove Albion', 0.1009213843641385),
 ('Newcastle United', 0.08983570647632995),
 ('Sheffield United', 0.04121093787078994),
 ('West Bromwich Albion', 0.027382524810271203)]

### Colley's ranking algorithm implementation

Constructing Colley's matrix:

In [ ]:
def colley(i,j, tms, tms_dict):
    teami = tms[i]
    games = tms_dict[teami]
    if i==j: return len(games)+2
    else:
        games = tms_dict[teami]
        teamj = tms[j]
        gamesij = [game['opponent'] for game in games if game['opponent']==teamj]
        return -len(gamesij)


def total_num_plays(tms, tms_dict):
    n = len(tms)
    M2 = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            M2[i,j] = colley(i, j, tms, tms_dict)
    return M2


total22 = total_num_plays(teams22, teams_dict22)
total21 = total_num_plays(teams21, teams_dict21)
print('Total number of plays for each team in SEASON 2021')
for elem in total21: print(*elem)
# print(*total21)
# print('Total number of plays for each team in SEASON 2022')
# print(total22)


Total number of plays for each team in SEASON 2021
24.0 0.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 26.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 24.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 24.0 -2.0 0.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 24.0 0.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 26.0 0.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 0.0 26.0 -2.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 0.0 0.0 24.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 24.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 0.0 0.0 -2.0 -2.0 24.0 -2.0 0.0 0.0 -2.0 -2.0 0.0 -2.0 -2.0 0.0 0.0
-2.0 0.0 -2.0 -2.0 -2.0 0.

Finding vector $\vec{b}$

In [ ]:
def colley_b(i, tms, tms_dict):
    games = tms_dict[tms[i]]
    n_wins = len([game for game in games if game['result']=='W'])
    n_losses = len([game for game in games if game['result']=='L'])
    return 1+0.5*(n_wins-n_losses)
b = np.array([colley_b(i, teams21, teams_dict21) for i in range(len(teams21))])

Solving a matrix equation to find the ranking vector $\vec{r}$

In [ ]:
vec = solve(total21,b)

vec = np.ndarray.flatten(abs(vec))
sorted_indices = vec.argsort()
ranked2 = [(teams21[i], vec[i]) for i in sorted_indices]
ranked2.reverse()
ranked2

[('Manchester City', 0.7692307692307692),
 ('Manchester United', 0.6153846153846152),
 ('West Ham United', 0.596153846153846),
 ('Chelsea', 0.5769230769230769),
 ('Leeds United', 0.5769230769230769),
 ('Aston Villa', 0.5576923076923076),
 ('Leicester City', 0.5384615384615384),
 ('Liverpool', 0.5384615384615383),
 ('Tottenham Hotspur', 0.5384615384615383),
 ('Everton', 0.5192307692307692),
 ('Wolverhampton Wanderers', 0.46153846153846145),
 ('Southampton', 0.44230769230769224),
 ('Brighton and Hove Albion', 0.44230769230769224),
 ('Arsenal', 0.4423076923076922),
 ('Burnley', 0.4423076923076922),
 ('Newcastle United', 0.423076923076923),
 ('Crystal Palace', 0.3076923076923076),
 ('Fulham', 0.2884615384615384),
 ('West Bromwich Albion', 0.24999999999999994),
 ('Sheffield United', 0.17307692307692302)]